In [22]:
import pandas as pd
import syft as sy
import numpy as np
from syft.core.adp.entity import Entity

In [23]:
data = pd.read_csv("../../trade_demo/datasets/ca - feb 2021.csv", low_memory=False)[0:10]
data.head()

,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,...,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Qty,Netweight (kg),Trade Value (US$),Flag
0,HS,2021,202102,February 2021,4,0,1,Imports,124,Canada,...,"Other Asia, nes",NaN,6117,"Clothing accessories; made up, knitted or croc...",0,NaN,NaN,NaN,9285,0
1,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Egypt,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,116604,0
2,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Kingdom,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,1495175,0
3,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Rep. of Tanzania,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,2248,0
4,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Singapore,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,47840,0


In [24]:
# Let's login into the domain
ca = sy.login(email="info@openmined.org", password="changethis", port=8081)

Connecting to http://localhost:8081... done! 	 Logging into canada... done!


In [ ]:
sy.networks[0].register_as_guest(ca) # adds my domain as a guest

In [28]:
ca.datasets[5]['Canada Trade']

<TensorWrappedSingleEntityPhiTensorPointer -> canada:11ab5df9e1384a8a882793755bdc5a1a>

In [5]:
# We will upload only the first few rows
# All these three columns are of `int` type

# NOTE: casting this tensor as np.int32 is REALLY IMPORTANT. We need to create flags for this or something
canada_trade = ((np.array(list(data['Trade Value (US$)'])) / 100000)[0:10]).astype(np.int32)
trade_partners = ((list(data['Partner'])))[0:10]

entities = list()
for i in range(len(trade_partners)):
    entities.append(Entity(name=trade_partners[i]))

# Upload a private dataset to the Domain object, as the root owner
sampled_canada_dataset = sy.Tensor(canada_trade)
sampled_canada_dataset.public_shape = sampled_canada_dataset.shape
sampled_canada_dataset = sampled_canada_dataset.private(0, 3, entity=entities[0]).tag("trade_flow")

In [1]:
import numpy as np

In [22]:
ca.load_dataset(assets={"Canada Trade": sampled_canada_dataset, "More Trade": data},
                name="ASdfasfasdf",
                description="asdfadsf")

Loading dataset... checking asset types...                                                                                                                                    

WARNING - Non-DP Asset: You just passed in a asset 'More Trade' which cannot be tracked with differential privacy because it is a <class 'pandas.core.frame.DataFrame'> object.

This means you'll need to manually approve any requests which leverage this data. If this is ok with you, proceed. If you'd like to use automatic differential privacy budgeting, please pass in a DP-compatible tensor type such as by calling .private() on a sy.Tensor with a np.int32 or np.float32 inside.


Are you sure you want to proceed? (y/n) y


Loading dataset... uploading... SUCCESS!                        

Run <your client variable>.datasets to see your new dataset loaded into your machine!


In [29]:
ca.datasets

Idx,Name,Description,Assets,Id
[0],Sample Dataset10,A nice description.,"[""Canada Trade""] -> Tensor[""More Trade""] -> Tensor",5c749798-c166-4ae5-8dc7-895a90399cc7
[1],Sample Dataset,A nice description.,"[""Canada Trade""] -> Tensor[""More Trade""] -> Tensor",52df3559-b9e1-4402-a7d9-161527a53efa
[2],Sample Dataset13,A nice description.,"[""Canada Trade""] -> Tensor[""More Trade""] -> DataFrame",29ef2aa0-99b8-4132-a198-40b7dc37dff0
[3],Sample Dataset3,A nice description.,"[""Canada Trade""] -> Tensor[""More Trade""] -> DataFrame",2d4aa206-4904-41cb-8744-2435b1334732
[4],Sample Dataset3,A nice description.,"[""Canada Trade""] -> Tensor[""More Trade""] -> DataFrame",c5b3c4ae-d3fd-4ea4-8d60-eaa99536a4b5
[5],ASdfasfasdf,asdfadsf,"[""Canada Trade""] -> Tensor[""More Trade""] -> DataFrame",8d618dea-7430-4b15-99df-b353577f090d
